In [6]:
import os
import random
import pickle
import zipfile
from tqdm import tqdm
import en_core_web_sm as en  #from the spaCy library, https://spacy.io/usage/

In [16]:
filename = './data/text8.zip'

In [17]:
def read_data(filename):
  with zipfile.ZipFile(filename) as f:
    text = tf.compat.as_str(f.read(f.namelist()[0]))
  return text
text = read_data(filename)

NameError: name 'tf' is not defined

In [9]:
def tokenize_text(text, nr_to_delete = 10):
    nlp = en.load()      #load the tokenizer
    
    batch_lenght = nlp.max_length
    nlp.max_length += 1
    text_tokenized = []  #every token in the original order
    tokens_count = {}    #a dict with everry word with his count
    vocab = []           #every unique word in the appearance order  

    batches = [text[x:x+batch_lenght] for x in range(0, len(text), batch_lenght)]  #place the text in n batchez of size batch_lenght

    del text  #delete the original text so we save memory

    for batch in tqdm(batches):  #tqdm is the progress bar
        tokens = nlp(batch)         
        for token in tokens:
            text_tokenized.append(token.string.strip())
            if text_tokenized[-1] in tokens_count:
                tokens_count[text_tokenized[-1]] += 1
            else:
                tokens_count[text_tokenized[-1]] = 1

    vocab = sorted(tokens_count, key=tokens_count.__getitem__) #get a list with the keys of the dict sorted by the nr of appearances
    
    sum = 0
    unk = vocab[:nr_to_delete]   #save a nr of words with the least appearances 
    for el in unk:               # delete the saved words and sum their appearence in sum
        sum += tokens_count[el]
        del tokens_count[el]
    tokens_count['UNK'] = sum    #replace the words with the "UNK" key
    
    vocab = sorted(tokens_count, key=tokens_count.__getitem__)  #resort he dict so we get the vocab without the words with the least appearences
    
    for idx,word in enumerate(text_tokenized): #replace the saved words with the "UNK" in the text
        for w in unk:
            if w == word:
                text_tokenized[idx] = 'UNK'             
    
    return text_tokenized,vocab       
    
tokenize_text, vocab = tokenize_text(text[:2500000])    

NameError: name 'text' is not defined